# Ejercicio de evaluación de Criptosistemas simétricos 

Alumna: Blanca Cano Camarero

## MiniAES

### Cuerpo en el que se trabaja 

In [2]:
F.<xi> = GF(2^4, modulus=GF(2)[x](x^4+x+1))
F.modulus()

x^4 + x + 1

In [3]:
# Definición de las respectivas funciones 

def PolinomioAPartirLista(lista):
    '''Transforma una lista en un coeficiente 
    La matriz es el coeficiente de orden 
    [c,b,a...]
    a + bx + cx^2 ...
    '''
    polinomio = lista[0]
    for coeficiente in lista[1:]:
        polinomio  = x*polinomio + coeficiente 
    return polinomio 

def Inverso(n):
    ''' n es una lista con el número en la base {x^3, x^2, x, 1}
    y devuelve su inverso o cero
    '''
    p = vector(n)
    if p != 0:
        p = vector(reversed(vector(F(vector(reversed(p)))^(-1))))
    return p

def Gamma(n):
    return list(Inverso(n)* matrix(GF(2),4,[0,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1]) + vector(GF(2),[0,0,1,1]))
    
def GammaMatrix(m):
    return [list(map(Gamma, row)) for row in m ]

def MixColumn(ll):
    A = matrix(F,2,[[x+1,x],[x,x+1]])

    b0 = ll[0][0]*x^3 + ll[0][1]*x^2 + ll[0][2]*x + ll[0][3]
    b1 = ll[1][0]*x^3 + ll[1][1]*x^2 + ll[1][2]*x + ll[1][3]
    b2 = ll[2][0]*x^3 + ll[2][1]*x^2 + ll[2][2]*x + ll[2][3]
    b3 = ll[3][0]*x^3 + ll[3][1]*x^2 + ll[3][2]*x + ll[3][3]
    
    
    B = matrix(F,2,[[b0,b2],[b1,b3]])

    return A*B

In [13]:
def nibblsub(ll):
    xx = vector(ll)
    if xx != 0:
        xx = vector(reversed(vector(F(vector(reversed(xx)))^(-1))))
    return list(xx*matrix(GF(2),4,[0,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1]) + vector(GF(2),[0,0,1,1]))
def key_schedule(kk): 
    ww= list(reversed(vector(GF(2),kk.bits())))
    while len(ww)<16:
        ww = [GF(2)(0)]+ww
    ww = [[ww[0],ww[1],ww[2],ww[3]],[ww[4],ww[5],ww[6],ww[7]],[ww[8],ww[9],ww[10],ww[11]],[ww[12],ww[13],ww[14],ww[15]]] 
    ww.append(list(vector(ww[0]) + vector(nibblsub(ww[3])) + vector(GF(2),[0,0,0,1]))) 
    ww.append(list(vector(ww[1]) + vector(ww[4])))
    ww.append(list(vector(ww[2]) + vector(ww[5])))
    ww.append(list(vector(ww[3]) + vector(ww[6])))                                                         
    ww.append(list(vector(ww[4]) + vector(nibblsub(ww[7])) + vector(GF(2),[0,0,1,0]))) 
    ww.append(list(vector(ww[5]) + vector(ww[8])))
    ww.append(list(vector(ww[6]) + vector(ww[9])))
    ww.append(list(vector(ww[7]) + vector(ww[10])))   
    return ww

def mixcolumn(ll):
    A = matrix(F,2,[[xi+1,xi],[xi,xi+1]])

    b0 = ll[0][0]*xi^3 + ll[0][1]*xi^2 + ll[0][2]*xi + ll[0][3]
    b1 = ll[1][0]*xi^3 + ll[1][1]*xi^2 + ll[1][2]*xi + ll[1][3]
    b2 = ll[2][0]*xi^3 + ll[2][1]*xi^2 + ll[2][2]*xi + ll[2][3]
    b3 = ll[3][0]*xi^3 + ll[3][1]*xi^2 + ll[3][2]*xi + ll[3][3]
    
    
    B = matrix(F,2,[[b0,b2],[b1,b3]])

    return A*B

def shiftrow(ll):
    return [ll[0],ll[3],ll[2],ll[1]]

def addroundkey(ll,kk): 
    return [list(vector(ll[0])+vector(kk[0])),list(vector(ll[1])+vector(kk[1])),list(vector(ll[2])+vector(kk[2])),list(vector(ll[3])+vector(kk[3]))]    

def NumberToMatriz (binario):
    '''Transforma un número binario de la forma 10001001 
    en una matriz 4x4, añade los ceros delante'''
    nl = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    n = list(map(int,reversed(str(binario))))
    n_length = len(n)
    ind = 3
    last = 0
    cnt = 0
    for ind in range(3,-1,-1):
        for i in range(0,4):
            if cnt < n_length:
                nl[ind][3-i] = n[cnt]
                cnt +=1
            else:
                return nl


def MiniAES1(kk,mm): #para m1 apartado 1
    ww = key_schedule(kk)
    estado = list(reversed(vector(GF(2),mm.bits())))
    while len(estado) < 32:
        estado = [GF(2)(0)] + estado
    
    estado = [[estado[0],estado[1],estado[2],estado[3]],[estado[4],estado[5],estado[6],estado[7]],[estado[8],estado[9],estado[10],estado[11]],[estado[12],estado[13],estado[14],estado[15]],[estado[16],estado[17],estado[18],estado[19]],[estado[20],estado[21],estado[22],estado[23]],[estado[24],estado[25],estado[26],estado[27]],[estado[28],estado[29],estado[30],estado[31]]] #FALTA CODIGO

    estado1 = list()
    estado1.append(estado[0])
    estado1.append(estado[1])
    estado1.append(estado[2])
    estado1.append(estado[3])
    estado = estado1
    print("m1 = ",estado)
    
    estado[3][3] = estado[3][3]  + 1
    c0 = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,1]]
    print("m1+c0 = ", estado)
    
    print(estado)
    print([ww[0],ww[1],ww[2],ww[3]])
    estado = addroundkey(estado,[ww[0],ww[1],ww[2],ww[3]])
    print("sigma_k0 = ",estado)
    
    estado = [nibblsub(ele) for ele in estado]
    print("gamma = ",estado)
    
    estado = shiftrow(estado)
    print("pi = ",estado)
    
    estado = mixcolumn(estado) 
    print("sacar coefs = ", estado)
    
def MiniAES1parte2(kk,estado): #para m1 apartado 1 
    ww = key_schedule(kk)
    print(estado)
    print([ww[4],ww[5],ww[6],ww[7]])
    estado = addroundkey(estado,[ww[4],ww[5],ww[6],ww[7]])
    print("sigma_k1 = ",estado)
    
    estado = [nibblsub(ele) for ele in estado]
    print("gamma =",estado)
    
    estado = shiftrow(estado)
    print("pi=",estado)
    
    estado = addroundkey(estado,[ww[8],ww[9],ww[10],ww[11]])
    print("sigma_k2 = ",estado)
    
    salida = list(reversed(estado[0]+estado[1]+estado[2]+estado[3]))
    
    return estado,Integer(salida,base=2).hex()


## Ejercicio 

Considerando el cifrado en bloques MiniAES descrito en el ejercicio 2.
1. Calcula $E_{dni}(0x01234567)$ usando el modo CBC e IV=$0x0001$.

Donde el mensaje a cifrar es $0x01234567$

In [23]:
mm = 0x01234567 #mensaje 
kk = 75577392 % 65536 # clave 
tam = 16 

# Calulamos k_o
ww =key_schedule(kk)

mensaje = mm.bits()
print(len(mensaje),len(mensaje)%tam, tam-len(mensaje)%16+len(mensaje)) 
# añadimos ceros hasta completar un bloque
longitud = 16-len(mensaje)%tam
for i in range(longitud+2, tam):
    mensaje.append(0)

mensaje = list(reversed(mensaje))
print(mensaje)
mc = 0
ms_length = len(mensaje)
c = 0b1
cnt = 0
indice_k_sigma = 0
for i in range(0, ms_length, tam):
    if(i == ms_length-tam):
        m = mensaje[i:]
    else:
        m = mensaje[i:tam]
    m='0b'+''.join(map(str, m))
    m = format(Integer(m), 'b')
    n = str(1^^Integer(m))
    nl = NumberToMatriz(n)
    ## Vamos concatenandando E_k
    # sigma k0
    sigma = addroundkey(nl,[ww[indice_k_sigma+0],ww[indice_k_sigma+1],ww[indice_k_sigma+2],ww[indice_k_sigma+3]])
    indice_k_sigma += 4
    print(f'sigma_k_{cnt}({nl}) = ', sigma)
    cnt +=1
    # gama 
    gamma = [nibblsub(ele) for ele in sigma]
    print(f'gamma{sigma}) = \n', gamma)
    # pi 
    pi = shiftrow(gamma)
    print(f'pi({gamma}) = \n', pi)
    # tetha miscolumn
    mc = [list(map(vector, row)) for row in mixcolumn(pi)] 
    print(f'mix column ({pi}) = \n', mc)
    # sigma k1
    sigma = addroundkey(mc,[ww[indice_k_sigma+0],ww[indice_k_sigma+1],ww[indice_k_sigma+2],ww[indice_k_sigma+3]])
    indice_k_sigma += 4
    print(f'sigma_k_{cnt}({nl}) = ', sigma)
    cnt +=1
    # gama 
    gamma = [nibblsub(ele) for ele in sigma]
    print(f'gamma{sigma}) = \n', gamma)
    # pi 
    pi = shiftrow(gamma)
    print(f'pi({gamma}) = \n', pi)
    # sigma k2
    sigma = addroundkey(pi,[ww[indice_k_sigma+0],ww[indice_k_sigma+1],ww[indice_k_sigma+2],ww[indice_k_sigma+3]])
    print(f'sigma_k_{cnt}({pi}) = ', sigma)







25 9 32
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]
sigma_k_0([[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0]]) =  [[0, 0, 1, 1], [1, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0]]
gamma[[0, 0, 1, 1], [1, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0]]) = 
 [[0, 1, 1, 1], [1, 1, 1, 0], [1, 0, 0, 0], [1, 1, 1, 1]]
pi([[0, 1, 1, 1], [1, 1, 1, 0], [1, 0, 0, 0], [1, 1, 1, 1]]) = 
 [[0, 1, 1, 1], [1, 1, 1, 1], [1, 0, 0, 0], [1, 1, 1, 0]]
mix column ([[0, 1, 1, 1], [1, 1, 1, 1], [1, 0, 0, 0], [1, 1, 1, 0]]) = 
 [[(0, 0, 1, 0), (0, 0, 1, 0)], [(0, 0, 1, 1), (0, 1, 0, 0)]]


TypeError: unable to find a common ring for all elements

In [18]:
vector(mc[0][0])

(1, 1, 1, 0)